## Setup

In [1]:
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

### Project Goal: to further understand the effectiveness of Devi Sansthan's 'Literacy Now' app by demographic features, and to pinpoint areas of improvement (in data collection) for future internal use

Developers: Vennila Annamalai (Data Science + Statistics, '25), Deepta Jasthi (Data Science + Business, '24)

Project Manager: Bella Chang (Data Science, '24)

## Data Cleaning

Tables that will be generated: 

1. Dataset on the user base (demographics) for students 

undefined. Dataset on the user base (demographics) for learners

undefined. Dataset on the grades (raw grade, max grade, final grade)

Side note: we initially tried to include a dataset on the campus data as well, but it seemed to lack structural organization (user_id column, which is supposed to be a primary key, is not unique). This is addressed in part II of the project as well.

In [2]:
student = pd.read_csv("/work/DATA/TSVs/mdl_student.txt", sep='\t')
learner = pd.read_csv("/work/DATA/TSVs/mdl_learner.txt", sep='\t')
grades = pd.read_csv("/work/DATA/CSVs/mdl_grade_grades.csv")
enrollments = pd.read_csv("/work/DATA/CSVs/mdl_user_enrolments.csv")
results = pd.read_csv("/work/DATA/CSVs/mdl_h5pactivity_attempts.csv")
campus = pd.read_csv("/work/DATA/CSVs/mdl_campus.csv")
school = pd.read_csv("/work/DATA/CSVs/mdl_school.csv")

In [3]:
student_filtered = student[["user_id", "campus_id", "mobile", "type", "name", "gender", "state", "city"]]
student_filtered["mobile"] = student_filtered["mobile"].fillna(-1).astype(int).astype(str)
student_filtered

/tmp/ipykernel_3107/46022674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_filtered["mobile"] = student_filtered["mobile"].fillna(-1).astype(int).astype(str)


,user_id,campus_id,mobile,type,name,gender,state,city
0,6,1,9450087881,student,piyusha sharma,female,uttar pradesh,lucknow
1,25,1,7894561237,student,student1 test,male,uttar pradesh,lucknow
2,35,1,9450087881,student,Rajan Yadav,male,NaN,NaN
3,36,1,9450087881,student,Rajan Yadav,male,NaN,NaN
4,65,0,9450087881,volunteer,rupapendra kumar,female,NaN,NaN
...,...,...,...,...,...,...,...,...
995,4122,3,-1,student,Sheikh Nabeezaahmad Ishtiyaq,female,31,589
996,4123,3,7985843650,student,Shivang Verma,male,31,589
997,4124,3,8957070066,student,Shivansh Dubey,male,31,589
998,4127,3,-1,student,Titiksha Dhyani,female,31,589


In [4]:
learner_filtered = learner[["user_id", "campus_id", "name", "age", "gender", "learner_location"]]
learner_filtered = learner_filtered[learner_filtered["age"] < 90]
learner_filtered

,user_id,campus_id,name,age,gender,learner_location
0,27,1,piyusha Sharma,10,female,Lko
1,37,1,Rohit Tiwari,12,male,Lucknow
2,64,1,Inshaa Fatima,10,other,Lucknow
3,67,0,Raju Raju,14,male,Lucknow
4,69,0,Ali Ali,15,male,Lucknow
...,...,...,...,...,...,...
13988,55761,23,Vinita Devi,12,female,Malviya nagar
13989,55765,8,Sarvansh Raj,10,male,Lucknow
13990,55768,23,Priti Majhi,20,female,Mumbai
13991,55770,11,Adarsh Kumar,13,male,"Kowabag goal park 279,b railway colony Gorakhpur"


In [5]:
results_grouped = results.groupby('userid').agg(np.mean)
grades_grouped = grades.groupby('userid').agg(np.mean)
result_merged = results_grouped.merge(grades_grouped, how = "inner", on="userid")

results_tbl = result_merged[["rawscore", "maxscore", "finalgrade", "duration", "rawgrade", "rawgrademax"]]
results_tbl = results_tbl.groupby("userid").agg(np.mean)
results_tbl = results_tbl.reset_index()
results_tbl = results_tbl.sort_values("userid")
results_tbl = results_tbl.rename(columns = {'userid': 'user_id'})
results_tbl

,user_id,rawscore,maxscore,finalgrade,duration,rawgrade,rawgrademax
0,64,22.666667,33.0,96.9695,313.0,96.9695,74.444444
1,67,32.000000,33.0,96.9700,167.0,96.9700,65.000000
2,361,26.000000,33.0,78.7880,586.0,78.7880,65.000000
3,1443,32.000000,33.0,96.9700,153.0,96.9700,93.333333
4,2907,32.000000,33.0,96.9700,455.0,96.9700,74.444444
...,...,...,...,...,...,...,...
1924,55715,9.000000,9.0,100.0000,90.0,100.0000,97.608696
1925,55721,67.000000,70.0,95.7140,406.0,95.7140,100.000000
1926,55735,54.000000,70.0,77.1430,203.0,77.1430,100.000000
1927,55741,83.000000,109.0,76.1470,332.0,76.1470,100.000000


In [6]:
learners_tbl = learner_filtered.sort_values("user_id")
learners_tbl

,user_id,campus_id,name,age,gender,learner_location
0,27,1,piyusha Sharma,10,female,Lko
1,37,1,Rohit Tiwari,12,male,Lucknow
2,64,1,Inshaa Fatima,10,other,Lucknow
3,67,0,Raju Raju,14,male,Lucknow
4,69,0,Ali Ali,15,male,Lucknow
...,...,...,...,...,...,...
13988,55761,23,Vinita Devi,12,female,Malviya nagar
13989,55765,8,Sarvansh Raj,10,male,Lucknow
13990,55768,23,Priti Majhi,20,female,Mumbai
13991,55770,11,Adarsh Kumar,13,male,"Kowabag goal park 279,b railway colony Gorakhpur"


In [7]:
student_filtered = student[["user_id", "campus_id", "mobile", "type", "name", "gender", "state", "city"]]
student_filtered["mobile"] = student_filtered["mobile"].fillna(-1).astype(int).astype(str)
student_filtered['mobile_area_codes'] = student_filtered['mobile'].str.slice(stop=2)

students_tbl = student_filtered.sort_values("user_id")
students_tbl['mobile_area_codes'] = students_tbl['mobile_area_codes'].astype('str')
students_tbl = students_tbl[students_tbl["type"] == "student"].drop(columns=['type'])
students_tbl

/tmp/ipykernel_3107/217024984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_filtered["mobile"] = student_filtered["mobile"].fillna(-1).astype(int).astype(str)
/tmp/ipykernel_3107/217024984.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student_filtered['mobile_area_codes'] = student_filtered['mobile'].str.slice(stop=2)


,user_id,campus_id,mobile,name,gender,state,city,mobile_area_codes
0,6,1,9450087881,piyusha sharma,female,uttar pradesh,lucknow,94
1,25,1,7894561237,student1 test,male,uttar pradesh,lucknow,78
2,35,1,9450087881,Rajan Yadav,male,NaN,NaN,94
3,36,1,9450087881,Rajan Yadav,male,NaN,NaN,94
7,75,1,9936231000,Garvit Halwasiya,male,NaN,NaN,99
...,...,...,...,...,...,...,...,...
995,4122,3,-1,Sheikh Nabeezaahmad Ishtiyaq,female,31,589,-1
996,4123,3,7985843650,Shivang Verma,male,31,589,79
997,4124,3,8957070066,Shivansh Dubey,male,31,589,89
998,4127,3,-1,Titiksha Dhyani,female,31,589,-1


## Part I: Visualizing effectiveness of app by demographic features

### a) Score of learners by gender

Upon an initial observation, we found that there are relatively similar distributions of mean grade among the different genders in the user base. However, there seems to be some slight differences as well (i.e. an abundance of outliers, a slightly wider distribution of ages among females). 

Possible uses of this for Devi Sansthan: understanding which genders are being served well in the app thus far, and which specific groups are not represented as well in the user base.

In [8]:
learner_result_tbl = learners_tbl.merge(results_tbl, how = "inner", on = "user_id")
learners_male = learner_result_tbl[learner_result_tbl["gender"] == "male"]
learners_male = learners_male["rawgrade"]

learners_female = learner_result_tbl[learner_result_tbl["gender"] == "female"]
learners_female = learners_female["rawgrade"]

learners_other = learner_result_tbl[learner_result_tbl["gender"] == "other"]
learners_other = learners_other["rawgrade"]

In [9]:
# Initial insight
age_learners = px.box(learner_result_tbl, x="rawgrade", color="gender", title = "Mean Raw Grade of Learners Sorted by Gender")
age_learners.show()

In [10]:
# Initial insight
age_learners = px.box(learner_result_tbl, x="finalgrade", color="gender", title = "Mean Final Grade of Learners Sorted by Gender")
age_learners.show()

In [11]:
px.violin(learner_result_tbl, y='rawgrade', color='gender', title = "Raw Grade Filtered by Gender")

There seems to be a relative difference of male learners' raw grades in comparison to female learners. Is it significant? We conducted a hypothesis test to see if it was.

Null: the mean raw grade between female and male students is roughly the same. Any difference is due to chance. 

Alternative: the mean raw grades between male and female students are not equal. A difference is due to something other than random chance.

In [38]:
#hypothesis test
learner_result_tbl = learners_tbl.merge(results_tbl, how = "inner", on = "user_id")

females = learner_result_tbl[learner_result_tbl['gender'] == 'female']['rawgrade']
males = learner_result_tbl[learner_result_tbl['gender'] == 'male']['rawgrade']

observed_test_statistic = np.abs(np.mean(females) - np.mean(males))

n_simulations = 1000

def bootstrap(group1, group2, n_simulations = 1000):
    simulated_test_statistics = []
    for i in range(n_simulations):
        # Resample with replacement from each group
        resampled_group1 = np.random.choice(group1, size=len(group1), replace=True)
        resampled_group2 = np.random.choice(group2, size=len(group2), replace=True)
        # Compute the test statistic for the resampled groups
        simulated_test_statistic = np.mean(resampled_group1) - np.mean(resampled_group2)
        simulated_test_statistics.append(np.abs(simulated_test_statistic))
    return simulated_test_statistics

simulated_test_statistics = bootstrap(females, males, 100)

p_value = np.sum(np.abs(simulated_test_statistics) >= np.abs(observed_test_statistic)) / len(simulated_test_statistics)
print("P value is:", p_value)

P value is: 0.98


In [13]:
fig = go.Figure(
    go.Histogram(x=simulated_test_statistics, 
                 marker=dict(color='rgba(12, 83, 148, 1)', line=dict(width=1, color='black')))
)
fig.update_layout(
    title="Histogram of Simulated Test Statistics",
    xaxis_title="Difference in Rawgrade Mean for Males and Females",
    yaxis_title="Frequency", showlegend=False
)
#plots.title('Prediction Made by the Null Hypothesis')
#plots.scatter(observed_test_statistic, 0, color='red', s=40);
fig.add_trace(go.Scatter(
    x=[observed_test_statistic], y=[0], mode='markers',
    marker=dict(color='red', size=10, symbol='x')
))

fig.show()

Because the p-value is well above a statistical cut-off of 0.05, we can conclude that the null is true: the mean final grade between male and female students is roughly the same. Therefore, the distribution and attentiveness towards each group of learners is going smoothly!

Key takeaways:

undefined. The raw grade distribution is largely skewed right for all genders, showing that most students are performing well when going through the lessons (largely within the 80-100 range).

undefined. There is no difference in grade between male and female learners. Thus, the lessons are being catered equally to both genders.

undefined. There seem to be a great deal of outliers in our data, showing that there is a wide range of learners within the app. Whether that is associated with a particular feature or cause could be a further point of exploration!

### b) Score of learners by geographical location

First, as a general contextual check, we wanted to see the age range of learners by geographical location.

In [14]:
area_codes = pd.read_csv("/work/DATA/CSVs/Book1.csv")
area_codes = area_codes.rename(columns = {'Area Code': 'mobile_area_codes'})
area_codes = area_codes.rename(columns = {'City': 'ci'})
area_codes['mobile_area_codes'] = area_codes['mobile_area_codes'].astype('str').str.slice(stop=2)

location_df = pd.merge(students_tbl, area_codes, on = "mobile_area_codes")

learner_loc_tbl = learners_tbl.merge(location_df, how = "inner", on = "name")
learner_loc_tbl = learner_loc_tbl.rename(columns = {"user_id_x" : "user_id"})

mean_age_tbl = learner_loc_tbl.groupby('ci').agg(np.mean).reset_index()
mean_age_tbl = mean_age_tbl[['ci', 'age']]
mean_age_tbl
#bootstrapping so the means are more accurate
n_bootstraps = 500
bootstrap_means = np.zeros((n_bootstraps, len(mean_age_tbl)))
for i in range(n_bootstraps):
    bootstrap_sample = mean_age_tbl.sample(frac=1, replace=True)
    bootstrap_means[i] = bootstrap_sample['age'].values
    for j, city in enumerate(bootstrap_sample['ci']):
        learner_loc_tbl.loc[learner_loc_tbl['ci'] == city, 'simulated_age'] = bootstrap_means[i][j]
ci_low = np.percentile(bootstrap_means, 2.5, axis=0)
ci_high = np.percentile(bootstrap_means, 97.5, axis=0)
for i in range(len(mean_age_tbl)):
    print(f"{mean_age_tbl['ci'][i]}: {mean_age_tbl['age'][i]:.2f}")
mean_age = px.bar(mean_age_tbl, y = "age", x = "ci", title = "Mean Age of Learners per Location")
mean_age.show()

Ahmadabad: 13.80
Amravati: 11.60
Bangalore: 10.67
Belgaum: 10.50
Bhilai: 6.89
Guntur: 9.33
Hubli-Dharwad: 10.50
Indore: 8.87
Jabalpur: 10.25
Kota: 12.40
Mangalore: 10.10
Mysore: 10.10
Raipur: 10.75
Vijayawada: 9.33
Visakhapatnam: 12.06
Warangal: 9.71


In [37]:
mean_age_cities_rounded = mean_age_tbl
mean_age_cities_rounded["age"] = mean_age_cities_rounded["age"].astype(int)
grouped_results = learner_result_tbl.groupby("age").mean().reset_index()
all_grades = grouped_results[grouped_results["age"].isin(mean_age_cities_rounded["age"])]
cities_grades = mean_age_cities_rounded.merge(all_grades[["age", "rawgrade", "finalgrade"]], on="age")

fig = go.Figure(data=[
    go.Bar(name='Raw Grade', x=cities_grades["ci"], y=cities_grades["rawgrade"]),
    go.Bar(name='Final Grade', x=cities_grades["ci"], y=cities_grades["finalgrade"])
])
fig.update_layout(title="Rough Raw and Final Grade per City")
fig.show()

### c) Score of learners by age

In [15]:
mean_result_tbl = learner_result_tbl.groupby('age').agg(np.mean).reset_index()
mean_result = px.scatter(mean_result_tbl, y = "rawgrade", x = "age", title = "Mean Raw Grade by Age")
mean_result.show()

In [16]:
mean_result_tbl = learner_result_tbl.groupby('age').agg(np.mean).reset_index()
mean_result = px.scatter(mean_result_tbl, y = "finalgrade", x = "age", title = "Mean Final Grade by Age")
mean_result.show()

In [17]:
mean_result_tbl = learner_result_tbl.groupby('age').agg(np.mean).reset_index()
mean_raw_grades = mean_result_tbl[["age", "rawgrade"]]
mean_raw_grades = mean_raw_grades.rename(columns={"rawgrade":"grade"})
mean_raw_grades["type"] = "raw"
mean_final_grades = mean_result_tbl[["age", "finalgrade"]]
mean_final_grades = mean_final_grades.rename(columns={"finalgrade":"grade"})
mean_final_grades["type"] = "final"

temp = mean_raw_grades.append(mean_final_grades)

temp_plot = px.scatter(temp, y='grade', x='age', color='type', title='Mean Grades (Raw and Final) by Age')
temp_plot.show()

In [18]:
learner_result_filtered = learner_result_tbl[learner_result_tbl["age"] < 20]

fig = px.density_contour(learner_result_filtered, x="age", y="rawgrade", title = "Density Plot for rawgrade by age")
fig.update_traces(contours_coloring="fill", contours_showlabels = True)
fig.show()

## Part II: improving data collection for future internal use

There were a few caveats that we believe should be improved within TWB x Devi Sansthan's data collection process to further simplify analysis and to consolidate the data more efficiently in the future:

1. Clarifying student and learner differences among the data in a way that is more intuitive to outside users; we were a bit unclear as to what the overlap was and how to distinguish between people who were actually using the app for learning and who were simply using it for their classroom or as a conductor for other learners.

undefined. Not having a table that maps campus_id to an actual campus. If there was another dataset that was able to just hold these fields (campus_id, campus, city) that would help to consolidate geographical data much more efficiently.

undefined. Having one primary key among all/most tables: 'user_id'. This created many problems down the line as some tables did not have that as their primary key (the campus table in particular) and made it difficult to merge datasets. Additionally, many students seemed to have the same name but different primary keys. This made things a bit confusing as analyzers.

undefined. Getting more comprehensive data about learners' scores: specifically, in each section of the app that they utilize (English, Hindi, math, etc.) This would help analysis greatly as it could see which areas users are struggling in, and perhaps in the long run, create personalized suggestions for users on where they could focus on while using the app.

undefined. Requiring all sections when gathering demographic data from the users. A lot of the users did not fill out some of the demographic data and that led to many NaN values that was difficult to incorporate into our analysis.

undefined. Standardizing collection by creating drop downs when collecting data from the users or another method that removes the concern of syntax issues in textual data - i.e. typos, mis-capitalization, etc. 

undefined. Having a metadata table to explain all features of tables. We ran into many confusing things, such as understanding the scale of "finalgrade" in comparison to the scale of "rawgrade" -- "finalgrade" ranges to 200 while "rawgrade" ranges to 100, which doesn't make implicit sense as we believed they would both be on the scale. Solving caveats like these could be improved by having one storage of metadata to supplement all features.

## Thank You!

Thank you for giving DSS @ Berkeley the opportunity to generate demographic reports for your app and organization! We were able to grow our data science and real-world business knowledge through you.

If you have any questions about the technical aspects of the project or encounter problems in the future, please email us at dss.berkeley@gmail.com. Feel free to contact any of the developers as well at vennila@berkeley.edu, jdeepta@berkeley.edu, and bellachang@berkeley.edu.

The main tools that we used in this project were:

- Pandas (for tables)

- Plotly (for generating visualizations)

- MySQL (for gathering the data)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=08b91cc6-deda-4616-92c9-e073a9a8b5c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>